# Estimating Mercury Accumulation in Lake Sediments and Glacial Input

This notebook estimates mercury (Hg) accumulation in the sediments of two alpine lakes (EYC and GDL) between 1970 and 2023. Calculations are based on:

- Mercury concentration (Hg)
- Dry bulk density (DBD)
- Sediment accumulation rate (SAR)

It also evaluates the total Hg mass in sediments and explores Hg input from glacial melt, using mass balance and published Hg concentrations in snow and ice.

In [12]:
import pandas as pd
import numpy as np

# === Function to compute Hg accumulation rate (HgAR) and error ===
def calculate_HgAR_vector(Hg_conc, density, SAR, err_Hg, err_DBD, err_SAR):
    HgAR = Hg_conc * density * SAR * 10
    rel_error = np.sqrt(err_Hg**2 + err_DBD**2 + err_SAR**2)
    abs_error = rel_error * HgAR
    return HgAR, abs_error

# === Function to integrate HgAR vector ===
def integrate_HgAR(HgAR_vector, age_vector):
    return np.trapz(HgAR_vector, x=age_vector)

# === Function to integrate propagated error ===
def integrate_error(HgAR_err_vector, age_vector):
    error_squared_sum = 0
    for i in range(len(age_vector) - 1):
        dx = age_vector.iloc[i+1] - age_vector.iloc[i]
        dy_err_i = HgAR_err_vector.iloc[i]
        dy_err_ip1 = HgAR_err_vector.iloc[i+1]
        trapezoid_var = ((dx / 2) ** 2) * (dy_err_i**2 + dy_err_ip1**2)
        error_squared_sum += trapezoid_var
    return np.sqrt(error_squared_sum)

# === Function to compute total Hg mass in lake sediments ===
def compute_mass(F_vector_area, err_area, lake_surface_m2):
    mass = F_vector_area * lake_surface_m2
    err_mass = err_area * lake_surface_m2
    return mass, err_mass

## Constants and Lake Surface Areas

We define the lake surface areas (in m²) and the assumed error on DBD:

- EYC: 0.151 km² = 151,000 m²
- GDL: 0.047 km² = 47,000 m²
- Relative error on dry bulk density: 5%

In [13]:
surface_GDL_m2 = 47000     # Grand Lac
surface_EYC_m2 = 151000    # Eychauda
err_DBD = 0.05

## Load Input Data

Three datasets are loaded:
- Hg.xlsx: Mercury concentrations and relative standard deviations
- DBD.xlsx: Dry bulk density profiles
- Age.xlsx: Age models and sediment accumulation rates

In [14]:
df_Hg = pd.read_excel('Hg.xlsx')
df_DBD = pd.read_excel('DBD.xlsx')
df_Age = pd.read_excel('210_Pb_dating/Age.xlsx')

## Extract Sedimentation Rates and Uncertainties

Sediment Accumulation Rates (SAR) and their relative errors are extracted from the first row.

In [15]:
SAR_EYC = df_Age['SAR_EYC'].iloc[0]
err_SAR_EYC = df_Age['err_SAR_EYC'].iloc[0]
SAR_GDL = df_Age['SAR_GDL'].iloc[0]
err_SAR_GDL = df_Age['err_SAR_GDL'].iloc[0]


## Calculate Hg Accumulation Rates and Uncertainties


$$
\mathrm{HgAR} = \mathrm{Hg} × \mathrm{DBD} × \mathrm{SAR} × 10
$$

(Conversion factor accounts for unit scaling.)

In [16]:
HgAR_EYC, err_EYC = calculate_HgAR_vector(
    df_Hg['Hg_conc_EYC'], df_DBD['DBD_EYC'], SAR_EYC,
    df_Hg['RSD_EYC'], err_DBD, err_SAR_EYC)

HgAR_GDL, err_GDL = calculate_HgAR_vector(
    df_Hg['Hg_conc_GDL'], df_DBD['DBD_GDL'], SAR_GDL,
    df_Hg['RSD_GDL'], err_DBD, err_SAR_GDL)

## Select Data from 1970 to 2023

HgAR values and corresponding ages are filtered to this time period.

In [17]:
age_EYC = df_Age['age_EYC']
age_GDL = df_Age['age_GDL']

HgAR_EYC_sel = HgAR_EYC.iloc[3:18]
age_EYC_sel = age_EYC.iloc[3:18]

HgAR_GDL_sel = HgAR_GDL.iloc[2:20]
age_GDL_sel = age_GDL.iloc[2:20]

# Ensure ascending time order
if age_EYC_sel.iloc[0] > age_EYC_sel.iloc[-1]:
    age_EYC_sel = age_EYC_sel[::-1]
    HgAR_EYC_sel = HgAR_EYC_sel[::-1]

if age_GDL_sel.iloc[0] > age_GDL_sel.iloc[-1]:
    age_GDL_sel = age_GDL_sel[::-1]
    HgAR_GDL_sel = HgAR_GDL_sel[::-1]

## Integrate Mercury Flux Over Time (µg/m²)

Numerical integration of the HgAR values from 1970–2023.

In [18]:
area_EYC = integrate_HgAR(HgAR_EYC_sel, age_EYC_sel)
area_GDL = integrate_HgAR(HgAR_GDL_sel, age_GDL_sel)

## Estimate Uncertainty on Integrated Flux

Propagated error is estimated using trapezoidal variance.

In [19]:
err_EYC_sel = err_EYC.iloc[0:18]
err_GDL_sel = err_GDL.iloc[0:20]

if age_EYC_sel.iloc[0] > age_EYC_sel.iloc[-1]:
    err_EYC_sel = err_EYC_sel[::-1]
if age_GDL_sel.iloc[0] > age_GDL_sel.iloc[-1]:
    err_GDL_sel = err_GDL_sel[::-1]

err_area_EYC = integrate_error(err_EYC_sel, age_EYC_sel)
err_area_GDL = integrate_error(err_GDL_sel, age_GDL_sel)

## Total Mercury Mass in Lake Sediments (kg)

Flux values are multiplied by lake surface to obtain mass estimates.

In [20]:
mass_EYC, err_mass_EYC = compute_mass(area_EYC, err_area_EYC, surface_EYC_m2)
mass_GDL, err_mass_GDL = compute_mass(area_GDL, err_area_GDL, surface_GDL_m2)

## Results and Output

Mercury mass and error are printed and saved to Excel for further use.

In [21]:
print(f"Hg flux 1970–2023:")
print(f"  EYC: {area_EYC:.2f} ± {err_area_EYC:.2f} µg/m²")
print(f"  GDL: {area_GDL:.2f} ± {err_area_GDL:.2f} µg/m²")

print(f"Estimated Mercury mass in sediments (1970–2023):")
print(f"  EYC lake: {mass_EYC / 1e9:.3f} kg ± {err_mass_EYC / 1e9:.3f} kg")
print(f"  GDL lake: {mass_GDL / 1e9:.3f} kg ± {err_mass_GDL / 1e9:.3f} kg")

results = pd.DataFrame({
    'Hg_AR_EYC': HgAR_EYC,
    'Err_EYC': err_EYC,
    'Hg_AR_GDL': HgAR_GDL,
    'Err_GDL': err_GDL
})

results.to_excel('HgAR.xlsx', index=False)
print("Results saved in 'HgAR.xlsx'")

Hg flux 1970–2023:
  EYC: 9633.87 ± 142.21 µg/m²
  GDL: 3467.79 ± 50.43 µg/m²
Estimated Mercury mass in sediments (1970–2023):
  EYC lake: 1.455 kg ± 0.021 kg
  GDL lake: 0.163 kg ± 0.002 kg
Results saved in 'HgAR.xlsx'
